### This note book runs the efficent model using the noisy students weights. We will use the features and fine tune it with our dataset.
### we do not have enough data for training it from scratch

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
from tensorflow import keras
import numpy as np
import pathlib
import cv2
import PIL
import PIL.Image
import time
import datetime
import matplotlib.pylab as plt
import tensorboard
SEEDS = 42
np.random.seed(SEEDS)
tf.random.set_seed(SEEDS)
import itertools
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
GPU = tf.config.list_physical_devices('GPU')
CPU= tf.config.list_physical_devices('CPU')
print("Num GPUs:", len(GPU))

print("Num CPUs:", len(CPU))

### set the memory growth incremental

In [ ]:
## The logical GPUs is not supported in the tensflow available binary, you need to compile it from scratch for multiple logical GPUs

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
      tf.config.experimental.set_memory_growth(gpus[0], True)
      print(len(gpus), "Physical GPUs,")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
model_name = "efficientnetv2-xl-21k-ft1k" # @param ['efficientnetv2-s', 'efficientnetv2-m', 'efficientnetv2-l', 'efficientnetv2-s-21k', 'efficientnetv2-m-21k', 'efficientnetv2-l-21k', 'efficientnetv2-xl-21k', 'efficientnetv2-b0-21k', 'efficientnetv2-b1-21k', 'efficientnetv2-b2-21k', 'efficientnetv2-b3-21k', 'efficientnetv2-s-21k-ft1k', 'efficientnetv2-m-21k-ft1k', 'efficientnetv2-l-21k-ft1k', 'efficientnetv2-xl-21k-ft1k', 'efficientnetv2-b0-21k-ft1k', 'efficientnetv2-b1-21k-ft1k', 'efficientnetv2-b2-21k-ft1k', 'efficientnetv2-b3-21k-ft1k', 'efficientnetv2-b0', 'efficientnetv2-b1', 'efficientnetv2-b2', 'efficientnetv2-b3', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'bit_s-r50x1', 'inception_v3', 'inception_resnet_v2', 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152', 'nasnet_large', 'nasnet_mobile', 'pnasnet_large', 'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

model_handle_map = {
  "efficientnetv2-s": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2",
  "efficientnetv2-m": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_m/feature_vector/2",
  "efficientnetv2-l": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/feature_vector/2",
  "efficientnetv2-s-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2",
  "efficientnetv2-m-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2",
  "efficientnetv2-l-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/feature_vector/2",
  "efficientnetv2-xl-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2",
  "efficientnetv2-b0-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2",
  "efficientnetv2-b1-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b1/feature_vector/2",
  "efficientnetv2-b2-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/feature_vector/2",
  "efficientnetv2-b3-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/feature_vector/2",
  "efficientnetv2-s-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_s/feature_vector/2",
  "efficientnetv2-m-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_m/feature_vector/2",
  "efficientnetv2-l-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_l/feature_vector/2",
  "efficientnetv2-xl-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2",
  "efficientnetv2-b0-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/feature_vector/2",
  "efficientnetv2-b1-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b1/feature_vector/2",
  "efficientnetv2-b2-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b2/feature_vector/2",
  "efficientnetv2-b3-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2",
  "efficientnetv2-b0": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2",
  "efficientnetv2-b1": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b1/feature_vector/2",
  "efficientnetv2-b2": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/feature_vector/2",
  "efficientnetv2-b3": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b3/feature_vector/2",
  "efficientnet_b0": "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
  "efficientnet_b1": "https://tfhub.dev/tensorflow/efficientnet/b1/feature-vector/1",
  "efficientnet_b2": "https://tfhub.dev/tensorflow/efficientnet/b2/feature-vector/1",
  "efficientnet_b3": "https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1",
  "efficientnet_b4": "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1",
  "efficientnet_b5": "https://tfhub.dev/tensorflow/efficientnet/b5/feature-vector/1",
  "efficientnet_b6": "https://tfhub.dev/tensorflow/efficientnet/b6/feature-vector/1",
  "efficientnet_b7": "https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1",
  "bit_s-r50x1": "https://tfhub.dev/google/bit/s-r50x1/1",
  "inception_v3": "https://tfhub.dev/google/imagenet/inception_v3/feature-vector/4",
  "inception_resnet_v2": "https://tfhub.dev/google/imagenet/inception_resnet_v2/feature-vector/4",
  "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/feature-vector/4",
  "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/feature-vector/4",
  "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/feature-vector/4",
  "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/feature-vector/4",
  "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/feature-vector/4",
  "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/feature-vector/4",
  "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/4",
  "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/feature_vector/4",
  "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/feature_vector/4",
  "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
  "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/4",
  "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4",
  "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5",
  "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5",
  "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
  "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5",
}

model_image_size_map = {
  "efficientnetv2-s": 384,
  "efficientnetv2-m": 480,
  "efficientnetv2-l": 480,
  "efficientnetv2-b0": 224,
  "efficientnetv2-b1": 240,
  "efficientnetv2-b2": 260,
  "efficientnetv2-b3": 300,
  "efficientnetv2-s-21k": 384,
  "efficientnetv2-m-21k": 480,
  "efficientnetv2-l-21k": 480,
  "efficientnetv2-xl-21k": 512,
  "efficientnetv2-b0-21k": 224,
  "efficientnetv2-b1-21k": 240,
  "efficientnetv2-b2-21k": 260,
  "efficientnetv2-b3-21k": 300,
  "efficientnetv2-s-21k-ft1k": 384,
  "efficientnetv2-m-21k-ft1k": 480,
  "efficientnetv2-l-21k-ft1k": 480,
  "efficientnetv2-xl-21k-ft1k": 512,
  "efficientnetv2-b0-21k-ft1k": 224,
  "efficientnetv2-b1-21k-ft1k": 240,
  "efficientnetv2-b2-21k-ft1k": 260,
  "efficientnetv2-b3-21k-ft1k": 300, 
  "efficientnet_b0": 224,
  "efficientnet_b1": 240,
  "efficientnet_b2": 260,
  "efficientnet_b3": 300,
  "efficientnet_b4": 380,
  "efficientnet_b5": 456,
  "efficientnet_b6": 528,
  "efficientnet_b7": 600,
  "inception_v3": 299,
  "inception_resnet_v2": 299,
  "nasnet_large": 331,
  "pnasnet_large": 331,
}

model_handle = model_handle_map.get(model_name)
pixels = model_image_size_map.get(model_name)

print(f"Selected model: {model_name} : {model_handle}")

IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 16
CLASS_NUM = 10

### defining training dataset diredtories

In [ ]:
def plot_hist(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,3.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
data_dir_1= "/home/pms/PMS-dataset/data-set/10-class/combined/training"
data_dir_2= "/home/pms/PMS-dataset/data-set/10-class/output/training"

data_dir_1 = pathlib.Path(data_dir_1)
data_dir_2 = pathlib.Path(data_dir_2)
print("Training-Directory-combined: ",data_dir_1)
print("Training-Directory-only-road: ",data_dir_1)


image_count_1 = len(list(data_dir_1.glob('*/*.jpg')))
image_count_2 = len(list(data_dir_2.glob('*/*.jpg')))
print(image_count_1)
print(image_count_2)

### preparing dataset from the directory into training and validation data

In [ ]:

train_ds_1 = tf.keras.utils.image_dataset_from_directory(
  str(data_dir_1),
  validation_split=0.2,
  subset="training",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)

train_ds_2 = tf.keras.utils.image_dataset_from_directory(
  str(data_dir_2),
  validation_split=0.2,
  subset="training",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)


val_ds_1 = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_dir_1),
  validation_split=0.2,
  subset="validation",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)

val_ds_2 = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_dir_2),
  validation_split=0.2,
  subset="validation",
  seed=SEEDS,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE
)


class_names = np.array(train_ds_1.class_names)
print(class_names)
class_names = np.array(train_ds_2.class_names)
print(class_names)
class_names_val = np.array(val_ds_1.class_names)
print(class_names_val)
class_names_val = np.array(val_ds_2.class_names)
print(class_names_val)
#concatenate the datasets
train_ds = train_ds_1.concatenate(train_ds_2)
val_ds = val_ds_1.concatenate(val_ds_2)

print(len(list(train_ds_1.as_numpy_iterator())))
print(len(list(train_ds_2.as_numpy_iterator())))
print(len(list(train_ds.as_numpy_iterator())))

In [ ]:
plt.figure(figsize=(2, 2))
for images, labels in train_ds.take(1):
  for i in range(4):
    ax = plt.subplot(2, 2, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### Apply Data augmentation to the training set and rescaling the images to 0-1 in both training and validation set and autotune

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomFlip('vertical'),
  tf.keras.layers.RandomRotation(2),
  tf.keras.layers.RandomZoom(.5),
  tf.keras.layers.RandomContrast(0.1, 0.2)
])
# apply normalization and augmentation to the training set and validation set
normalization_layer = tf.keras.layers.Rescaling(1./255)

## WARNING: the rescaling layer is not applied if uisng keras EfficientNet application model not in tensorhub model

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# check the batch size
image_batch, label_batch = next(iter(train_ds.take(1)))
print(image_batch.shape, label_batch.shape)

In [ ]:
## for debugging purpose
#tf.debugging.set_log_device_placement(True) # this line is to turn on debugging information

train_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy", patience=3, restore_best_weights=True
    )
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [ ]:
with tf.device('device:GPU:0'):
    do_fine_tuning = False

    print("Building model with", model_handle)
    model = tf.keras.Sequential([
        # Explicitly define the input shape so the model can be properly
        # loaded by the TFLiteConverter
        tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
        hub.KerasLayer(model_handle, trainable=do_fine_tuning),
        tf.keras.layers.Dropout(rate=0.2),
        #tf.keras.layers.Dense(CLASS_NUM,kernel_regularizer=tf.keras.regularizers.l2(0.0001))
        tf.keras.layers.Dense(CLASS_NUM,kernel_regularizer=tf.keras.regularizers.l2(0.0001),activation='softmax')
    ])
    model.build((None,)+IMAGE_SIZE+(3,))
    model.summary()

In [ ]:
with tf.device('device:GPU:0'):
  base_learning_rate = 0.01
## choose the optimizer uncomment the one you want

  #model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
  #              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  #              metrics=['accuracy'])

  model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=base_learning_rate, momentum=0.9), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
with tf.device('device:GPU:0'):
    NUM_EPOCHS = 1
# class weight are calculated by the total of samples divided by the number of classes and the number of samples in each class
    class_weights = {0: 0.85,1 : 0.83,2 : 1.495,3 : 1.57,4 : 2.153,5 : 2.53,6 : 1.17,7 : 0.89,8 : 0.609,9 : 0.54}
    history = model.fit(
        train_ds,
        epochs=NUM_EPOCHS,
        validation_data=val_ds,
        callbacks=tensorboard_callback,
        class_weight=class_weights
    )

plot_hist(history)

In [ ]:
with tf.device('device:GPU:0'):
  base_learning_rate = 0.001
  model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=base_learning_rate, momentum=0.9), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

  NUM_EPOCHS = 200
  history = model.fit(
          train_ds,
          epochs=NUM_EPOCHS,
          validation_data=val_ds,
          callbacks=tensorboard_callback,
          class_weight=class_weights)

plot_hist(history) 

In [ ]:
reloaded = model
with tf.device('device:GPU:0'):
  test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/training')
  print(test_data_dir)
  img_height = pixels
  img_width = pixels
  batch_size = 8
  class_number = CLASS_NUM

  test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_data_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
    # normalize the images is necessary for models in keras-tensorflow-hub
  test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

  loss, accuracy = reloaded.evaluate(test_dataset)
  print('Test accuracy :', accuracy)
  print('Test loss :', loss)

In [ ]:
model.save('pms_model/tensoflow-class-10-efficientnetv2-xl-21k-ft1k')

In [ ]:
import xlwt
from xlwt import Workbook
  
# Workbook is created
wb = Workbook()
# add_sheet is used to create sheet.
test = wb.add_sheet('Test-road-Summary')

In [ ]:
with tf.device('device:GPU:0'):
    result_test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/testing')
    output_test_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/results/output_test-efficientnetv2-xl-21k-ft1k-with-weight')
    try:
        os.makedirs(output_test_dir, exist_ok=True)
        print("Directory created")
    except OSError as e:
        print("Directory already exists")
count = 0
countf = 0
countF = 0
countT = 0
count_loop = 0
count1 = 0
count2 = 0
count3 = 0
count4 = 0
count5 = 0
count6 = 0
count7 = 0
count8 = 0
count9 = 0
count10 = 0

root, class_n, files = next(os.walk(result_test_data_dir))
print(class_n)
for c in class_n:
    new_root = os.path.join(output_test_dir, c)
    try:
        os.mkdir(new_root)
    except OSError as e:
        print("Directory already exists")
    for dirs in os.walk(os.path.join(root, c)):
        for files in dirs:
            for filename in files:
                if(filename.endswith(".jpg")):
                    path = os.path.join(root, c, filename)
                    #img = tf.keras.utils.load_img(path, target_size=(img_height, img_width))
                    orig = cv2.imread(path)
                    img = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (img_height, img_width))
                    img_array = tf.keras.utils.img_to_array(img)
                    img_array = tf.expand_dims(img_array, 0) # Create a batch
                    img_array = normalization_layer(img_array)
                    with tf.device('device:GPU:0'):
                        predictions = reloaded.predict(img_array)
                        score = tf.nn.softmax(predictions[0])
                    if (int(class_names[np.argmax(score)])==int(c)):
                        count=count+1
                    elif(int(class_names[np.argmax(score)]) ==10):
                        countf=countf+1
                        count10=count10+1
                    elif(int(class_names[np.argmax(score)])==9):
                        countf=countf+1
                        count9=count9+1
                    elif(int(class_names[np.argmax(score)]) ==8):
                        countf=countf+1
                        count8=count8+1
                    elif(int(class_names[np.argmax(score)]) ==7):
                        countf=countf+1
                        count7=count7+1
                    elif(int(class_names[np.argmax(score)]) ==6):
                        countf=countf+1
                        count6=count6+1
                    elif(int(class_names[np.argmax(score)]) ==5):
                        countf=countf+1
                        count5=count5+1
                    elif(int(class_names[np.argmax(score)]) ==4):
                        countf=countf+1
                        count4=count4+1
                    elif(int(class_names[np.argmax(score)]) ==3):
                        countf=countf+1
                        count3=count3+1
                    elif(int(class_names[np.argmax(score)])==2):
                        countf=countf+1
                        count2=count2+1
                    elif(int(class_names[np.argmax(score)])==1):
                        countf=countf+1
                        count1=count1+1
                    else:
                        print('error')
                    new_filename = filename.split(".")[0]+ filename.split(".")[1] + "_" + "P_" + str(int(class_names[np.argmax(score)])) + ".jpg"
                    new_path = os.path.join(new_root, new_filename)
                    cv2.imwrite(new_path, orig)
                    #print(new_filename )
    countT = countT + count
    countF = countF + countf
    print('=========================================================')
    print('class: ',c, 'Images: ',len(files), 'TP: ', count/len(files), 'FP: ',countf/len(files))
    #print('one : ',count1/len(files),'two: ',count2/len(files),'three: ',count3/len(files),'four: ',count4/len(files),'five: ',count5/len(files),'six: ',count6/len(files),'seven: ',count7/len(files),'eight: ',count8/len(files),'nine: ',count9/len(files),'ten: ',count10/len(files))
    print('one: ',count1,'two: ',count2,'three: ',count3,'four: ',count4,'five: ',count5,'six: ',count6,'seven: ',count7,'eight: ',count8,'nine: ',count9,'ten: ',count10)
    test.write(int(c), 1, count1)
    test.write(int(c), 2, count2)
    test.write(int(c), 3, count3)
    test.write(int(c), 4, count4)
    test.write(int(c), 5, count5)
    test.write(int(c), 6, count6)
    test.write(int(c), 7, count7)
    test.write(int(c), 8, count8)
    test.write(int(c), 9, count9)
    test.write(int(c), 10, count10)
    test.write(int(c), 11, count)
    test.write(int(c), 12, countf)

    count = 0
    countf = 0
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    count6 = 0
    count7 = 0
    count8 = 0
    count9 = 0
    count10 = 0
    count_loop = count_loop + 1
print('Total classes: ',count_loop,'Total TP: ', countT/(countT+countF), 'Total FP: ',countF)
test.write(11, 0, countT/(countT+countF))
test.write(11, 1, countF)

In [ ]:
wb.save('tenclass-road-efficientnetv2-xl-21k-ft1k-with-weight.xls')

In [ ]:
# Workbook is created
wb2 = Workbook()
# add_sheet is used to create sheet.
test2 = wb2.add_sheet('Test-Summary-road')
test2.write(0,0,'No')
test2.write(0,1,'filename')
test2.write(0,2,'true-class')
test2.write(0,3,'predicted-class')
iter = 1

with tf.device('device:GPU:0'):
    result_test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/testing')
    #result_test_data_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/5-class/output/testing')
    output_test_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/results/test')
    #output_test_dir = pathlib.Path('/home/pms/PMS-dataset/data-set/10-class/output/results/test')
    try:
        os.makedirs(output_test_dir, exist_ok=True)
        print("Directory created")
    except OSError as e:
        print("Directory already exists")
count = 0
countf = 0
countF = 0
countT = 0
count_loop = 0
count1 = 0
count2 = 0
count3 = 0
count4 = 0
count5 = 0

root, class_n, files = next(os.walk(result_test_data_dir))
print(class_n)
for c in class_n:
    new_root = os.path.join(output_test_dir, c)
    try:
        os.mkdir(new_root)
    except OSError as e:
        print("Directory already exists")
    for dirs in os.walk(os.path.join(root, c)):
        for files in dirs:
            for filename in files:
                if(filename.endswith(".jpg")):
                    path = os.path.join(root, c, filename)
                    #img = tf.keras.utils.load_img(path, target_size=(img_height, img_width))
                    orig = cv2.imread(path)
                    img = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (img_height, img_width))
                    img_array = tf.keras.utils.img_to_array(img)
                    img_array = tf.expand_dims(img_array, 0) # Create a batch
                    img_array = normalization_layer(img_array)
                    
                    predictions = reloaded.predict(img_array)
                    score = tf.nn.softmax(predictions[0])
                    ## write to excel------------------------------------
                    test2.write(iter, 0, iter)
                    test2.write(iter, 1, filename)
                    test2.write(iter, 2, int(c))
                    test2.write(iter, 3, int(class_names[np.argmax(score)]))
                    iter = iter + 1 # this iterator is for serial number in excel file to count number of jpg files
                        #-----------------------------------------------------
                    if (int(class_names[np.argmax(score)])==int(c)):
                        count=count+1
                    elif(int(class_names[np.argmax(score)]) ==5):
                        countf=countf+1
                        count5=count5+1
                    elif(int(class_names[np.argmax(score)]) ==4):
                        countf=countf+1
                        count4=count4+1
                    elif(int(class_names[np.argmax(score)]) ==3):
                        countf=countf+1
                        count3=count3+1
                    elif(int(class_names[np.argmax(score)])==2):
                        countf=countf+1
                        count2=count2+1
                    elif(int(class_names[np.argmax(score)])==1):
                        countf=countf+1
                        count1=count1+1
                    elif(int(class_names[np.argmax(score)])==6):
                        countf=countf+1
                        count6=count6+1
                    elif(int(class_names[np.argmax(score)])==7):
                        countf=countf+1
                        count7=count7+1
                    elif(int(class_names[np.argmax(score)])==8):
                        countf=countf+1
                        count8=count8+1
                    elif(int(class_names[np.argmax(score)])==9):
                        countf=countf+1
                        count9=count9+1
                    elif(int(class_names[np.argmax(score)])==10):
                        countf=countf+1
                        count10=count10+1
                    else:
                        print('error')
                    new_filename = filename.split(".")[0]+ filename.split(".")[1] + "_" + "P_" + str(int(class_names[np.argmax(score)])) + ".jpg"
                    new_path = os.path.join(new_root, new_filename)
                    cv2.imwrite(new_path, orig)
                    #print(new_filename )

    countT = countT + count
    countF = countF + countf
    print('=========================================================')
    print('class: ',c, 'Images: ',len(files), 'TP: ', count/len(files), 'FP: ',countf/len(files))
    print('one: ',count1,'two: ',count2,'three: ',count3,'four: ',count4,'five: ',count5)

    count = 0
    countf = 0
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    count6 = 0
    count7 = 0
    count8 = 0
    count9 = 0
    count10 = 0
    count_loop = count_loop + 1
print('Total classes: ',count_loop,'Total TP: ', countT/(countT+countF), 'Total FP: ',countF)

wb2.save('road-10-classes.xls')